In [1]:
from RunGerryChainDefs import *

In [2]:
(graph, chain, gdf) = GerryChainSetUp(Input_Shp='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/State Shp Files/AL/Alabama_VTD_District_Intersection.shp',
                                        Total_Steps=10)
(chain_list, metadata_list, part_dict, seg_output) = RunGerryChain(GDF=gdf, Graph=graph, Chain=chain, VTD_Data='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Agregate VTD Demographic Data/AL.csv',
                    Checkpoint = 2, Print_Iterations = False, 
                    Output_Csv = '/Users/rebeccawagner/Desktop/csv', 
                    Output_Metadata = '/Users/rebeccawagner/Desktop/meta',
                    Output_Seg='/Users/rebeccawagner/Desktop/seg')

    

/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column last!
  warnings.wa

0
Empty DataFrame
Columns: []
Index: [01001, 01002, 01003, 01004, 01005, 01006, 01007]
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0

Chain complete. Time elapsed: 5.024320125579834 seconds.
0


In [3]:
import pandas as pd


In [4]:
pd.read_csv("/Users/rebeccawagner/Desktop/seg_10all")

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10
0,1001,-0.449858,18.550356,18.550356,18.550356,18.550356,18.550356,0.358454,3.070203,3.070203,3.070203
1,1002,-7.150571,-7.150571,-4.876559,-8.571489,-3.834800,-7.021390,-7.021390,-7.021390,-7.021390,-8.584359
2,1003,-3.792898,-3.792898,-6.062259,-2.364924,-7.112741,-3.919550,-3.919550,-3.919550,-3.919550,-2.374068
3,1004,-10.025746,-10.025746,-10.025746,-10.025746,-10.025746,-10.025746,-10.025746,-10.025746,1.028736,1.028736
4,1005,0.198728,0.198728,0.198728,0.198728,0.198728,0.198728,0.198728,0.198728,-10.830530,-10.830530
5,1006,-14.170530,-14.170530,-14.170530,-14.170530,-14.170530,-14.170530,4.315877,1.627208,1.627208,1.627208
6,1007,18.428387,-0.531932,-0.531932,-0.531932,-0.531932,-0.531932,-0.531932,-0.531932,-0.531932,-0.531932


In [82]:
# Initializing variables
count = 0# recording iteration count
chain_list = []# blank list to vtd assignment for each iteration
metadata_list = []# blank list to store metadata
tot_num_edges = graph.number_of_edges() #will be used to calculate proportion of edges not cut

chain_list_all = []
metadata_list_all = []
seg_df = pd.DataFrame(index=gdf["cd_117"].unique())
print(seg_df)


Empty DataFrame
Columns: []
Index: [01001, 01002, 01003, 01004, 01005, 01006, 01007]


In [83]:
Output_Seg = '/Users/rebeccawagner/Desktop/seg'

for partition in chain:
    count += 1
    # bad_incumbents = 0
    part_dict = {"Iteration": count}

    meta_dict = {"Iteration": count}

    # Storing all the metadata. I tried to find a better way to code this, but I ended up just hardcoding it in.
    # Dictionary key will be the column name in your metadata output, so you can make it however readable you want it to be
    meta_dict["incumbent_issues"] = partition.bad_incumbents
    meta_dict["prop_uncut_edges"] = 1 - len(partition["cut_edges"])/tot_num_edges
    meta_dict["county_splits"] = partition.split_counter
    meta_dict["num_split_counties"] = partition.num_split_counties
    meta_dict["oversplit_counties"] = partition.oversplit_counties
    meta_dict["black_majority"] = partition.subpop_dists

    # Extracting and storing assingment for each VTD
    for nodenum in range(len(partition.graph.nodes)):
        part_dict[str(partition.graph.nodes[nodenum]['vtd_cd_117'])] = partition.assignment[nodenum]

    chain_list.append(part_dict)

    metadata_list.append(meta_dict)

    chain_list_all.append(part_dict)
    metadata_list_all.append(meta_dict)

    if count % 2 == 0:
        seg_output = SegregationOutput(VTD_Data='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Agregate VTD Demographic Data/AL.csv',
                            Chain_List=chain_list,Output_Seg =Output_Seg)
        seg_df = seg_df.merge(right=seg_output, how='outer', left_index=True, right_index=True)

        chain_list = []
        Output_Seg = Output_Seg[:-2] + '_' + str(count)

    

In [84]:
seg_df

,1,2,3,4,5,6,7,8,9,10
01001,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858
01002,-7.150571,-7.150571,13.616242,13.616242,-1.314229,1.430968,-9.647545,0.279969,0.279969,0.279969
01003,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898,-3.792898
01004,-10.025746,-14.304117,-14.304117,-14.304117,0.793507,-1.949734,-1.949734,-1.949734,1.964712,1.964712
01005,0.198728,0.198728,0.198728,-9.756985,-9.756985,-9.756985,1.322962,1.322962,-2.663873,-2.367323
01006,-14.170530,-9.885108,-9.885108,0.174259,0.174259,0.174259,0.174259,-9.822343,-9.822343,-10.076204
01007,18.428387,18.428387,-2.438269,-2.438269,-2.438269,-2.438269,-2.438269,-2.438269,-2.438269,-2.438269


In [75]:
seg_output = SegregationOutput(VTD_Data='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Agregate VTD Demographic Data/AL.csv',
                        Chain_List=chain_list,Output_Seg=Output_Seg)

In [78]:
seg_output

,1,2,3,4,5,6,7,8,9,10
01001,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858
01002,-7.150571,-7.150571,-7.150571,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996
01003,-3.792898,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.327127,-3.327127
01004,-10.025746,-10.025746,-10.529075,-10.529075,12.477924,-11.687483,-9.213845,5.051157,5.051157,-9.810779
01005,0.198728,0.198728,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005
01006,-14.170530,-14.564225,-14.564225,-14.564225,-14.564225,9.808148,7.372643,-6.896417,-6.926764,-6.926764
01007,18.428387,18.428387,18.428387,13.532642,-9.595155,-9.595155,-9.595155,-9.595155,-9.595155,5.285079


In [79]:
seg_df

""
01001
01002
01003
01004
01005
01006
01007


In [80]:
seg_df.merge(right=seg_output, left_index=True, right_index=True)


,1,2,3,4,5,6,7,8,9,10
01001,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858,-0.449858
01002,-7.150571,-7.150571,-7.150571,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996,-2.340996
01003,-3.792898,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.352147,-3.327127,-3.327127
01004,-10.025746,-10.025746,-10.529075,-10.529075,12.477924,-11.687483,-9.213845,5.051157,5.051157,-9.810779
01005,0.198728,0.198728,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005,0.789005
01006,-14.170530,-14.564225,-14.564225,-14.564225,-14.564225,9.808148,7.372643,-6.896417,-6.926764,-6.926764
01007,18.428387,18.428387,18.428387,13.532642,-9.595155,-9.595155,-9.595155,-9.595155,-9.595155,5.285079


In [5]:
pd.read_csv("/Users/rebeccawagner/Desktop/Csv_100_200_300_400_500_600_700_800_900_100000all")

KeyboardInterrupt: 

In [16]:
import csv

with open(file='/Users/rebeccawagner/Desktop/Csv_100_200_300_400_500_600_700_800_900_100000all') as f:
    reader = csv.reader(f)
    my_list = [row[-1] for row in reader]

with open(file='/Users/rebeccawagner/Desktop/Csv_100_200_300_400_500_600_700_800_900_100000all') as f:
    reader = csv.reader(f)
    list_1 = [row[0] for row in reader]

In [46]:
shp_file = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/State Shp Files/AL/Alabama_VTD_District_Intersection.shp'
al_gdf = gpd.read_file(shp_file) # returns a GeoDataFrame from file
al_gdf["incumbent"] = al_gdf["incumbent"].fillna(0)
al_gdf = al_gdf.drop(columns="cd_117")

In [47]:
al_gdf

,OBJECTID,Join_Count,TARGET_FID,VTD,vtd_cd_117,dups,total,one_race,wht_1,blk_1,...,last,first,middle,race,res_ad_lat,res_ad_lon,incumbent,Shape_Le_1,Shape_Area,geometry
0,1,0,0,01003000001,01003000001_01001,0,357,334,256,59,...,None,None,None,None,0.0,0.0,0,1.004504,0.024391,"POLYGON ((-87.76515 31.29735, -87.76360 31.297..."
1,2,0,1,01003000002,01003000002_01001,0,251,242,98,139,...,None,None,None,None,0.0,0.0,0,1.012120,0.022135,"POLYGON ((-87.81299 31.13709, -87.81653 31.131..."
2,3,0,2,01003000003,01003000003_01001,0,558,538,260,270,...,None,None,None,None,0.0,0.0,0,0.597227,0.010159,"POLYGON ((-87.90042 30.93770, -87.90144 30.937..."
3,4,0,3,01003000004,01003000004_01001,0,1175,1119,821,275,...,None,None,None,None,0.0,0.0,0,0.641053,0.012414,"POLYGON ((-87.92361 30.93146, -87.92348 30.931..."
4,5,0,4,01003000005,01003000005_01001,0,1632,1578,401,1148,...,None,None,None,None,0.0,0.0,0,0.188839,0.000877,"POLYGON ((-87.77521 30.83709, -87.77556 30.837..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1840,0,1839,01131005006,01131005006_01007,0,508,505,39,465,...,None,None,None,None,0.0,0.0,0,0.141377,0.000727,"POLYGON ((-87.31821 32.02500, -87.31764 32.024..."
1840,1841,0,1840,01131006001,01131006001_01007,0,429,420,145,275,...,None,None,None,None,0.0,0.0,0,0.394519,0.005829,"POLYGON ((-87.55981 32.05385, -87.55962 32.052..."
1841,1842,0,1841,01131006002,01131006002_01007,0,1242,1217,480,733,...,None,None,None,None,0.0,0.0,0,0.819720,0.011736,"POLYGON ((-87.55691 32.02856, -87.55576 32.028..."
1842,1843,0,1842,01131006004,01131006004_01007,0,208,208,16,192,...,None,None,None,None,0.0,0.0,0,0.448974,0.007100,"POLYGON ((-87.62858 31.94270, -87.62749 31.940..."


In [53]:
last_iter = pd.DataFrame(list_1).merge(right=pd.DataFrame(my_list), right_index=True, left_index=True)
last_iter = last_iter.drop(labels=0, axis=0)
last_iter['vtd'] = [x[:-6] for x in last_iter["0_x"]]
last_iter = last_iter.drop(columns="0_x")
last_iter = last_iter.rename({"0_y":"cd_117"}, axis=1)
last_iter

,cd_117,vtd
1,01003,01003000001
2,01003,01003000002
3,01003,01003000003
4,01003,01003000004
5,01003,01003000005
...,...,...
1840,01004,01131005006
1841,01004,01131006001
1842,01004,01131006002
1843,01004,01131006004


In [54]:
al_gdf = al_gdf.merge(right=last_iter, left_on='VTD', right_on='vtd')

In [55]:
al_gdf

,OBJECTID,Join_Count,TARGET_FID,VTD,vtd_cd_117,dups,total,one_race,wht_1,blk_1,...,middle,race,res_ad_lat,res_ad_lon,incumbent,Shape_Le_1,Shape_Area,geometry,cd_117,vtd
0,1,0,0,01003000001,01003000001_01001,0,357,334,256,59,...,None,None,0.0,0.0,0,1.004504,0.024391,"POLYGON ((-87.76515 31.29735, -87.76360 31.297...",01003,01003000001
1,2,0,1,01003000002,01003000002_01001,0,251,242,98,139,...,None,None,0.0,0.0,0,1.012120,0.022135,"POLYGON ((-87.81299 31.13709, -87.81653 31.131...",01003,01003000002
2,3,0,2,01003000003,01003000003_01001,0,558,538,260,270,...,None,None,0.0,0.0,0,0.597227,0.010159,"POLYGON ((-87.90042 30.93770, -87.90144 30.937...",01003,01003000003
3,4,0,3,01003000004,01003000004_01001,0,1175,1119,821,275,...,None,None,0.0,0.0,0,0.641053,0.012414,"POLYGON ((-87.92361 30.93146, -87.92348 30.931...",01003,01003000004
4,5,0,4,01003000005,01003000005_01001,0,1632,1578,401,1148,...,None,None,0.0,0.0,0,0.188839,0.000877,"POLYGON ((-87.77521 30.83709, -87.77556 30.837...",01003,01003000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,1840,0,1839,01131005006,01131005006_01007,0,508,505,39,465,...,None,None,0.0,0.0,0,0.141377,0.000727,"POLYGON ((-87.31821 32.02500, -87.31764 32.024...",01004,01131005006
1854,1841,0,1840,01131006001,01131006001_01007,0,429,420,145,275,...,None,None,0.0,0.0,0,0.394519,0.005829,"POLYGON ((-87.55981 32.05385, -87.55962 32.052...",01004,01131006001
1855,1842,0,1841,01131006002,01131006002_01007,0,1242,1217,480,733,...,None,None,0.0,0.0,0,0.819720,0.011736,"POLYGON ((-87.55691 32.02856, -87.55576 32.028...",01004,01131006002
1856,1843,0,1842,01131006004,01131006004_01007,0,208,208,16,192,...,None,None,0.0,0.0,0,0.448974,0.007100,"POLYGON ((-87.62858 31.94270, -87.62749 31.940...",01004,01131006004
